In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pickle,sys,os
import antropy as ant
from numpy import *
import heapq, qnorm
import ruptures as rpt
from scipy.stats import norm, pearsonr, zscore, rankdata
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../../Human_hg38_chrom_ends.pkl','rb') as a:
    ENDS = pickle.load(a)

with open('../../Human_hg38_auto_10kb_windows.pkl','rb') as b:
    W = pickle.load(b)

In [4]:
def get_numpy_arr(infile,columns=None):
    arr_file = load(infile)
    if columns is None:
        arr = arr_file['arr']
    else:
        arr = arr_file['arr'][:,columns]
    for cols in range(arr.shape[1]):
        csum = sum(arr[:,cols])
        arr[:,cols] /= csum
    return arr

def get_full_table(arr,win=10000):
    rows = 0
    full_table = []
    for ch in range(22):
        chrom = 'chr'+str(ch+1)
        bound = ENDS[chrom]
        windows = bound // win
        full_table.append(arr[rows:rows+windows,:])
        rows += windows
    if rows!=arr.shape[0]:
        print('error')
    return full_table

def get_full_table_k(full_table,win=10000,k=1):
    if k == 1:
        return full_table
    else:
        ft = []
        for ch in range(22):
            chrom = 'chr'+str(ch+1)
            bound = ENDS[chrom]
            win2 = win*k
            windows2 = bound // win2
            arr_i = full_table[ch]
            arr_k = zeros((windows2,arr_i.shape[1]))
            for j in range(windows2-1):
                arr_k[j] = arr_i[j*k:(j+1)*k,:].sum(axis=0)
            arr_k[-1] = arr_i[(windows2-1)*k:,:].sum(axis=0)
            ft.append(arr_k)
        return ft

def get_full_table_array_k(full_table,win=10000,k=1):
    if k == 1:
        for ch in range(22):
            if ch == 0:
                arr = full_table[ch]
            else:
                arr = concatenate((arr,full_table[ch]),axis=0)
    else:
        ft = []
        for ch in range(22):
            chrom = 'chr'+str(ch+1)
            bound = ENDS[chrom]
            win2 = win*k
            windows2 = bound // win2
            arr_i = full_table[ch]
            arr_k = zeros((windows2,arr_i.shape[1]))
            for j in range(windows2-1):
                arr_k[j] = arr_i[j*k:(j+1)*k,:].sum(axis=0)
            arr_k[-1] = arr_i[(windows2-1)*k:,:].sum(axis=0)
            if ch==0:
                arr = arr_k
            else:
                arr = concatenate((arr,arr_k),axis=0)
            #ft.append(arr_k)
    return arr

def get_full_table_array_k_chr(full_table,win=10000,k=1):
    ends = []
    if k == 1:
        for ch in range(22):
            if ch == 0:
                arr = full_table[ch]
            else:
                arr = concatenate((arr,full_table[ch]),axis=0)
            ends.append(arr.shape[0])
    else:
        ft = []
        for ch in range(22):
            chrom = 'chr'+str(ch+1)
            bound = ENDS[chrom]
            win2 = win*k
            windows2 = bound // win2
            arr_i = full_table[ch]
            arr_k = zeros((windows2,arr_i.shape[1]))
            for j in range(windows2-1):
                arr_k[j] = arr_i[j*k:(j+1)*k,:].sum(axis=0)
            arr_k[-1] = arr_i[(windows2-1)*k:,:].sum(axis=0)
            if ch==0:
                arr = arr_k
            else:
                arr = concatenate((arr,arr_k),axis=0)
            ends.append(arr.shape[0])
    return arr, ends

def get_rate_of_change(table):
    for ch in range(22):
        arr = table[ch]
        if ch == 0:
            new_arr = arr[1:,:] - arr[:-1,:]
        else:
            arr = arr[1:,:] - arr[:-1,:]
            new_arr = concatenate((new_arr,arr),axis=0)
    for cols in range(new_arr.shape[1]):
        sigma = sqrt(sum([x**2 for x in new_arr[:,cols]])/new_arr.shape[0])
        if sigma == 0:
            print('samples with all zero values found')
        new_arr[:,cols] /= sigma
    print(new_arr.shape)
    return new_arr

def get_rate_of_change_chr(table,ch):
    arr = table[ch]
    new_arr = arr[1:,:] - arr[:-1,:]
    for cols in range(new_arr.shape[1]):
        sigma = sqrt(sum([x**2 for x in new_arr[:,cols]])/new_arr.shape[0])
        new_arr[:,cols] /= sigma
    #print(new_arr.shape)
    return new_arr

def get_arr_for_chrs(infile,columns=None,k=1):
    df_arr = get_numpy_arr(infile,columns)
    table = get_full_table(df_arr)
    ftk = get_full_table_k(table,k=k)
    tables = []
    for ch in range(22):
        arr = get_rate_of_change_chr(ftk,ch)
        tables.append(arr)
    return tables

def get_arr(infile,columns=None,k=1):
    df_arr = get_numpy_arr(infile,columns)
    table = get_full_table(df_arr)
    ftk, ends = get_full_table_array_k_chr(table,k=k)
    #print(ends)
    return ftk #arrs

In [5]:
k = 1
thgs = get_arr('merged_table_no_correction_1000g_10kb.npz',k=k)
thgs.shape

(287487, 3202)

In [6]:
ctrl38s = get_arr('../../merged_table_no_correction_ctrl38_10kb.npz',k=k)
ctdna36s = get_arr('../../merged_table_no_correction_ctdna36luad_10kb.npz',k=k)
brcas = get_arr('../../merged_table_no_correction_brca_10kb.npz',k=k)
crcs = get_arr('../../merged_table_no_correction_crc_10kb.npz',k=k)
ctrl29s = get_arr('../../merged_table_no_correction_ctrl29_10kb.npz',k=k)
ega8460 = get_arr('../../merged_table_no_correction_EGA8460_10kb.npz',k=k)
ega5343_ctrl = get_arr('../../merged_table_no_correction_EGA5343_ctrl_10kb.npz',k=k)

In [7]:
ega5343_prostate = get_arr('../../merged_table_no_correction_EGA5343_prostate_10kb.npz',k=k)

In [8]:
arr_file = load('../../merged_table_no_correction_EGA7080_10kb.npz')
samples = arr_file['samples']

ctrl = [i for i in range(len(samples)) if samples[i].startswith('Ctrl')]
ctdna = [i for i in range(len(samples)) if i not in ctrl] # and samples[i].endswith('_1')
ega7080_ctrl = get_arr('../../merged_table_no_correction_EGA7080_10kb.npz',columns=ctrl,k=k)
ega7080_ctdna = get_arr('../../merged_table_no_correction_EGA7080_10kb.npz',columns=ctdna,k=k)

In [9]:
ends = [24895, 49114, 68943, 87964, 106117, 123197,139131, 153644, 167483, 180862, 
        194370, 207697, 219133, 229837, 240036, 249069, 257394, 265431, 271292, 277736, 282406, 287487]
feats = [i for i in range(thgs.shape[0]) if sum(thgs[i,:])>0]
print(ends[-1]-len(feats))
#zf = [i for i in range(thgs.shape[0]) if i not in feats]
#print(len(zf))
chr_splits = [0]
j = 0
for e in ends:
    for i in range(j,len(feats)):
        if feats[i] > e:
            chr_splits.append(i)
            j = i
            break
chr_splits.append(len(feats))
print(chr_splits)

51248
[0, 21838, 43007, 61151, 77732, 93356, 108799, 122508, 135682, 146854, 158478, 170121, 181337, 190291, 197701, 205137, 211316, 216730, 222093, 226049, 231029, 233445, 236239]


In [10]:
def get_rank(arr):
    ranked_arr = []
    for i in range(arr.shape[1]):
        ranked = rankdata(arr[:,i],'min')
        ranked_arr.append(ranked)
    return array(ranked_arr).T

In [11]:
rctrl38s = get_rank(ctrl38s[feats,:])
rctdna36s = get_rank(ctdna36s[feats,:])
rpost22s = get_rank(post22s[feats,:])
rbrcas = get_rank(brcas[feats,:])
rcrcs = get_rank(crcs[feats,:])
rctrl29s = get_rank(ctrl29s[feats,:])
rega5343_ctrl = get_rank(ega5343_ctrl[feats,:])
rega7080_ctrl = get_rank(ega7080_ctrl[feats,:])
rega8460 = get_rank(ega8460[feats,:])
rega7080_ctdna = get_rank(ega7080_ctdna[feats,:])
rthgs = get_rank(thgs[feats,:])

In [12]:
rega5343_prostate = get_rank(ega5343_prostate[feats,:])

In [14]:
rctrls = hstack((rega5343_ctrl,rega7080_ctrl,rctrl38s,rctrl29s))

In [15]:
rctdnas = hstack((rega7080_ctdna,rctdna36s,rbrcas,rcrcs,rega8460))

In [16]:
# segmentation
def get_breakpoints(arr=rthgs,splits=chr_splits):
    bkps = {}
    breaks = [0]
    offset = 0
    for ch in range(22):
        ctrl_signals = mean(arr[splits[ch]:splits[ch+1],:],axis=1)
        sigma = std(ctrl_signals)
        n = len(ctrl_signals)
        if n < 100:
            offset+=n
            continue
        algo = rpt.Binseg(model="l1",min_size=100).fit(ctrl_signals) # min size is 1Mbp # "l1", "rbf", "linear", "normal", "ar",...
        my_bkps = algo.predict(pen=log(n)*sigma)
        bkps['chr'+str(ch+1)] = [offset]+[x+offset for x in my_bkps]
        breaks += [x+offset for x in my_bkps]
        offset+= n
    return bkps, breaks
bkps, breaks = get_breakpoints()

In [17]:
def get_segment_mean(arr,breaks=breaks):
    n = len(breaks)-1
    arr_seg = []
    for i in range(n):
        arr_seg.append(mean(arr[breaks[i]:breaks[i+1],:],axis=0))
    return array(arr_seg)

In [18]:
sctrl38s = get_segment_mean(rctrl38s)
sctrl29s = get_segment_mean(rctrl29s) 
sega7080_ctrl = get_segment_mean(rega7080_ctrl)
sega5343_ctrl = get_segment_mean(rega5343_ctrl)
print(median(std(sctrl38s,axis=1)),median(std(sctrl29s,axis=1)),median(std(sega7080_ctrl,axis=1)),median(std(sega5343_ctrl,axis=1)))

14766.613891687324 9091.224919564898 8910.747007094713 5086.423075596531


In [80]:
nsegments = []
for j in range(1,23):
    chrom = 'chr'+str(j)
    nsegments.append((chrom,len(bkps[chrom])-1))
print(nsegments)

[('chr1', 98), ('chr2', 100), ('chr3', 78), ('chr4', 81), ('chr5', 75), ('chr6', 71), ('chr7', 67), ('chr8', 67), ('chr9', 48), ('chr10', 56), ('chr11', 56), ('chr12', 41), ('chr13', 32), ('chr14', 35), ('chr15', 36), ('chr16', 28), ('chr17', 20), ('chr18', 30), ('chr19', 21), ('chr20', 21), ('chr21', 12), ('chr22', 17)]


In [20]:
def get_chrom_mean(arr,splits=chr_splits):
    n = len(splits)-1
    arr_seg = []
    for i in range(n):
        arr_seg.append(mean(arr[splits[i]:splits[i+1],:],axis=0))
    return array(arr_seg)

def sort_arr_by_chr_mean(arr,chrom='chr1',splits=chr_splits):
    i = int(chrom.split('hr')[1])-1
    m = mean(arr[splits[i]:splits[i+1],:],axis=0)
    sorted_idx = argsort(m)[::-1]
    return sorted_idx

In [21]:
def get_rank_hist(arr, nbins=8):
    hist = []
    for s in range(arr.shape[1]):
        h,_ = histogram(arr[:,s], bins=linspace(0,240000,nbins+1))
        hist.append(h)
    return array(hist).T

def test_gaussian(arr,ctrl,error=1e-2): # feats x n
    estimator = EllipticEnvelope(random_state=12345,contamination=error,support_fraction=1.0).fit(ctrl.T)
    return estimator.predict(arr.T)

In [22]:
def chr_hist_segment_classifier(arr,chrom,ctrls=rega7080_ctrl,error=1e-2,nbins=8,bps=bkps,splits=chr_splits):
    sorted_idx = sort_arr_by_chr_mean(arr,chrom)
    sorted_ctrl_idx = sort_arr_by_chr_mean(ctrls,chrom)
    sarr = get_segment_mean(arr[:,sorted_idx],breaks=bps[chrom])
    sctrl = get_segment_mean(ctrls[:,sorted_ctrl_idx],breaks=bps[chrom])
    harr = get_rank_hist(sarr,nbins)#;print(mean(harr,axis=1).shape)
    hctrl = get_rank_hist(sctrl,nbins)
    arr_labels = test_gaussian(harr,hctrl,error=error)
    return arr_labels[sorted_idx]

In [23]:
def ensemble_classifier_sample(arr,ctrls,incl=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr12','chr13','chr15'],alpha=0.05,error=1e-64,nbins=8):#
    arr_labels = zeros(arr.shape[1])
    count = 0
    for chrom in incl:#bkps:
        nseg = len(bkps[chrom])-1
        labels = array([max(x,0) for x in chr_hist_segment_classifier(arr,chrom,ctrls,error,nbins)])
        labels *= nseg
        arr_labels += labels
        count += nseg
    arr_labels/=count
    return len([x for x in arr_labels if x<1-alpha])/len(arr_labels)

In [72]:
def ensemble_classifier(arr,ctrls,incl=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr12','chr13','chr15'],alpha=0.05,error=1e-64,nbins=8,verbose=False):#
    arr_labels = zeros(arr.shape[1])
    count = 0
    for chrom in incl:#bkps:
        nseg = len(bkps[chrom])-1
        labels = array([max(x,0) for x in chr_hist_segment_classifier(arr,chrom,ctrls,error,nbins)])
        labels *= nseg
        arr_labels += labels
        count += nseg
    arr_labels/=count
    print("total number of segments:",count)
    print("proportion of outliers in arr:",len([x for x in arr_labels if x<1-alpha])/len(arr_labels))

def tuning(ctrls,a0=0.05,a1=0.25):
    alpha = a0; alphas = []; spes = []
    while alpha < a1:
        count = 0
        for i in range(ctrls.shape[1]):
            count += ensemble_classifier_sample(ctrls[:,i].reshape(-1,1),ctrls[:,[j for j in range(ctrls.shape[1]) if j!=i]],nbins=8,alpha=alpha)
        spe = 1-count/ctrls.shape[1]
        alphas.append(alpha); spes.append(spe)
        alpha += 0.01
    plt.scatter(alphas,spes)

def autotuning(ctrls,specificity=0.9,a0=0.05,a1=0.25):
    alpha = a0; alphas = []; spes = []
    while alpha < a1:
        count = 0
        for i in range(ctrls.shape[1]):
            count += ensemble_classifier_sample(ctrls[:,i].reshape(-1,1),ctrls[:,[j for j in range(ctrls.shape[1]) if j!=i]],nbins=8,alpha=alpha)
        spe = 1-count/ctrls.shape[1]
        alphas.append(alpha); spes.append(spe)
        if spe>=specificity:
            print('alpha:',alpha)
            print('leave-one-out-specificity:',spe)
            return alpha
        else:
            alpha += 0.01
    best_spe = max(spes)
    for i,a in enumerate(alphas):
        if spes[i]==best_spe:
            print('alpha:',a)
            print('leave-one-out-specificity:',best_spe)
            return a

def outlier_detection(arr,ctrls,specificity=0.9,condition_matched=True,a0=None,a1=None):
    if condition_matched:
        if a0 is None or a1 is None:
            a0 = 0.05; a1=0.5
    else: # if condition unmatched, consider larger alpha (0.1,0.15,0.2) and/or larger leave-one-out specificity to tolerate batch effects
        if a0 is None or a1 is None:
            a0 = 0.15; a1 = 0.5
    alpha = autotuning(ctrls,specificity=specificity,a0=a0,a1=a1)
    ensemble_classifier(arr,ctrls,alpha=alpha)

In [27]:
outlier_detection(rega7080_ctdna,rega7080_ctrl)

alpha: 0.15
leave-one-out-specificity: 0.9090909090909091
total number of segments: 679
proportion of outliers in arr: 0.970954356846473


In [53]:
outlier_detection(rega7080_ctdna,hstack((rega5343_ctrl,rctrl29s,rctrl38s)),specificity=0.95,condition_matched=False,a0=0.1,a1=0.5)
outlier_detection(rega7080_ctrl,hstack((rega5343_ctrl,rctrl29s,rctrl38s)),specificity=0.95,condition_matched=False,a0=0.1,a1=0.5)

alpha: 0.1
leave-one-out-specificity: 0.9560439560439561
total number of segments: 679
proportion of outliers in arr: 0.6846473029045643
alpha: 0.1
leave-one-out-specificity: 0.9560439560439561
total number of segments: 679
proportion of outliers in arr: 0.045454545454545456


In [26]:
outlier_detection(rctdna36s,rctrl38s)

alpha: 0.05
leave-one-out-specificity: 0.9210526315789473
total number of segments: 679
proportion of outliers in arr: 0.4444444444444444


In [62]:
outlier_detection(rctdna36s,hstack((rega5343_ctrl,rctrl29s,rega7080_ctrl)),specificity=0.95,condition_matched=False)
outlier_detection(rctrl38s,hstack((rega5343_ctrl,rctrl29s,rega7080_ctrl)),specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.96
total number of segments: 679
proportion of outliers in arr: 0.25
alpha: 0.15
leave-one-out-specificity: 0.96
total number of segments: 679
proportion of outliers in arr: 0.2894736842105263


In [69]:
outlier_detection(rega5343_prostate,rega5343_ctrl)

alpha: 0.47000000000000025
leave-one-out-specificity: 0.9166666666666666
total number of segments: 679
proportion of outliers in arr: 1.0


In [71]:
outlier_detection(rega5343_prostate,hstack((rctrl38s,rctrl29s,rega7080_ctrl)),specificity=0.95,condition_matched=False)
outlier_detection(rega5343_ctrl,hstack((rctrl38s,rctrl29s,rega7080_ctrl)),specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.9662921348314607
total number of segments: 679
proportion of outliers in arr: 1.0
alpha: 0.15
leave-one-out-specificity: 0.9662921348314607
total number of segments: 679
proportion of outliers in arr: 0.0


In [73]:
outlier_detection(rctrl29s,hstack((rctrl38s,rega5343_ctrl,rega7080_ctrl)),specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.9642857142857143
total number of segments: 679
proportion of outliers in arr: 0.0


In [74]:
outlier_detection(rbrcas,rctrls,specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.9734513274336283
total number of segments: 679
proportion of outliers in arr: 1.0


In [75]:
outlier_detection(rcrcs,rctrls,specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.9734513274336283
total number of segments: 679
proportion of outliers in arr: 1.0


In [76]:
outlier_detection(rega8460,rctrls,specificity=0.95,condition_matched=False)

alpha: 0.15
leave-one-out-specificity: 0.9734513274336283
total number of segments: 679
proportion of outliers in arr: 1.0


In [81]:
# 1kGP contamination test
df = pd.read_csv('3202_sequence_index.tsv',sep='\t')
runs = df['RUN_NAME'].tolist()
files = df['LIBRARY_NAME'].tolist()
sorted_files = sorted(files)
batches = [x.split('-')[-2] for x in runs]
bs = unique(batches)
BATCHES = {}
for batch in bs:
    idxs = [i for i in range(len(batches)) if batches[i]==batch]
    samples = array(files)[idxs]
    BATCHES[batch] = list(samples)
# ten unselected batches, batch1 for training, batch2-9 for testing 
batch1 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[1]]]
batch2 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[2]]]
batch3 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[3]]]
batch4 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[4]]]
batch5 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[5]]]
batch6 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[6]]]
batch7 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[7]]]
batch8 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[8]]]
batch9 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[9]]]
batch10 = rthgs[:,[i for i in range(len(batches)) if batches[i]==bs[10]]]

In [85]:
def contamination_test(chrom,test_set=hstack((batch2,batch3,batch4,batch5,batch6,batch7,batch8,batch9,batch10)),
                       train_set=batch1,error=1e-32,bps=bkps,splits=chr_splits,nbins=8):
    sorted_idx = sort_arr_by_chr_mean(test_set,chrom)
    sorted_ctrl_idx = sort_arr_by_chr_mean(train_set,chrom)
    sarr = get_segment_mean(test_set[:,sorted_idx],breaks=bps[chrom])
    sctrl = get_segment_mean(train_set[:,sorted_ctrl_idx],breaks=bps[chrom])
    harr = get_rank_hist(sarr,nbins)#;print(mean(harr,axis=1).shape)
    hctrl = get_rank_hist(sctrl,nbins)
    ctrl_labels = test_gaussian(hctrl,hctrl,error=error)
    arr_labels = test_gaussian(harr,hctrl,error=error)
    print("contamination rate for "+chrom+":",len([x for x in arr_labels if x==-1])/len(arr_labels))

In [87]:
for ch in range(22):
    chrom = 'chr'+str(ch+1)
    contamination_test(chrom)

contamination rate for chr1: 0.008583690987124463
contamination rate for chr2: 0.008583690987124463
contamination rate for chr3: 0.0
contamination rate for chr4: 0.004291845493562232
contamination rate for chr5: 0.017167381974248927
contamination rate for chr6: 0.0
contamination rate for chr7: 0.0
contamination rate for chr8: 0.07725321888412018
contamination rate for chr9: 0.27467811158798283
contamination rate for chr10: 0.2575107296137339
contamination rate for chr11: 0.34763948497854075
contamination rate for chr12: 0.02145922746781116
contamination rate for chr13: 0.012875536480686695
contamination rate for chr14: 0.21888412017167383
contamination rate for chr15: 0.017167381974248927
contamination rate for chr16: 0.004291845493562232
contamination rate for chr17: 0.0
contamination rate for chr18: 0.03862660944206009
contamination rate for chr19: 0.012875536480686695
contamination rate for chr20: 0.0
contamination rate for chr21: 0.08583690987124463
contamination rate for chr22: 0.

In [91]:
print(batch1.shape[1],batch2.shape[1],batch3.shape[1],batch4.shape[1],batch5.shape[1],
      batch6.shape[1],batch7.shape[1],batch8.shape[1],batch9.shape[1],batch10.shape[1])

26 26 26 27 26 26 25 25 26 26
